In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import os
import json
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
print(tf.version.VERSION)

1.15.0


In [0]:
IMG_SIZE = 512
NUM_IMGS = 10
task_path = "drive/My Drive/test_task/"
data_path = task_path + "data/"
checkpoints_path = task_path + "checkpoints/"

In [4]:
descr = json.load(open(data_path + "label_descriptions.json", "rb"))
label_names = [cat["name"] for cat in descr["categories"]]
NUM_CLASSES = len(descr['categories']) + 1 # plus background
print(f"Number of attributes : {len(descr['attributes'])}")
print(f"Number of categoriees : {len(descr['categories'])}")
print(f"Number of classes to predict : {NUM_CLASSES}")

Number of attributes : 92
Number of categoriees : 46
Number of classes to predict : 47


In [0]:
df = pd.read_csv(data_path + "10images.csv")
df["ClassId"] = df["ClassId"].apply(lambda x: int(x.split('_')[0]) + 1) # use only categories as classes

In [0]:
def get_images(df):
  # get each image (as multiple segments) in a saparate dataframe
  imgs = []
  for img_name in df["ImageId"].unique():
    imgs.append(df.loc[df["ImageId"] == img_name])
  return imgs

In [0]:
def segments2labeled(segments):
  # converts dataframe with segments to labeled image
  h, w = segments["Height"].iloc[0], segments["Width"].iloc[0]

  flat = np.full(h * w, 0)
  for index, row in segments.iterrows():
    pixels = [int(p) for p in row["EncodedPixels"].split()]
    
    class_id = row["ClassId"] # int(row["ClassId"].split('_')[0]) + 1

    for i in range(0, len(pixels), 2):
      start = pixels[i] - 1
      flat[start: start + pixels[i+1]] = class_id
    
  labeled_img = flat.reshape((h, w), order='F')
  final_img = cv2.resize(labeled_img, 
                         (IMG_SIZE, IMG_SIZE), 
                         interpolation=cv2.INTER_NEAREST)
  return final_img

In [0]:
def labeled2masked(labeled):
  # converts labeled 2d image to 3d array of masks
  labels = np.unique(labeled)
  labels = labels[labels != 0] # except zero (background)
  rois = list()
  mask = np.zeros((*labeled.shape, len(labels)))
  for i, label in enumerate(labels):
    segment = labeled == label
    mask[:, :, i][segment] = 1

    mask_pos = np.where(segment) # find locations of boundary boxes
    y1, x1 = np.min(mask_pos, axis=1)
    y2, x2 = np.max(mask_pos, axis=1)

    rois.append([y1, x1, y2, x2])

  return mask, labels, np.array(rois)

Save reshaped and annotated images in two folders (for PSPNet)


In [0]:
for image_segments in get_images(df):
  img_name = image_segments["ImageId"].iloc[0]
  annotated_img = segments2labeled(image_segments) # labeled == annotated
  png_name = img_name.split('.')[0] + ".png"
  cv2.imwrite(f"{data_path}annotated/{png_name}", annotated_img)

  img = cv2.imread(f"{data_path}{NUM_IMGS}images/{img_name}")
  reshaped_img = cv2.resize(img, 
                            (IMG_SIZE, IMG_SIZE), 
                            interpolation=cv2.INTER_NEAREST)
  cv2.imwrite(data_path + f"10reshaped/{png_name}", reshaped_img)

In [10]:
!git clone https://www.github.com/divamgupta/image-segmentation-keras.git
os.chdir("image-segmentation-keras")

fatal: destination path 'image-segmentation-keras' already exists and is not an empty directory.


In [11]:
from keras_segmentation.models.model_utils import transfer_weights
from keras_segmentation.pretrained import pspnet_50_ADE_20K
from keras_segmentation.models.pspnet import pspnet_50

Using TensorFlow backend.


In [12]:
pretrained_model = pspnet_50_ADE_20K()
pspnet = pspnet_50(n_classes=NUM_CLASSES)

transfer_weights(pretrained_model, pspnet)
# pspnet.summary()

0it [00:00, ?it/s]

Copying weights 


225it [01:54,  1.96it/s]

Copied weights of 120 layers and skipped 1 layers


In [13]:
pspnet.train(
    train_images =  "../" + data_path + "10reshaped/",
    train_annotations =  "../" + data_path + "annotated/",
    checkpoints_path = "../" + checkpoints_path, 
    epochs=1
)

100%|██████████| 10/10 [00:00<00:00, 85.51it/s]


Verifying training dataset
Dataset verified! 
Starting Epoch  0
Epoch 1/1
512/512 [==============================] - 235s 458ms/step - loss: 0.0741 - acc: 0.9828
saved  ../drive/My Drive/test_task/checkpoints/.model.0
Finished Epoch 0


In [0]:
pspnet.save_weights(f"../{checkpoints_path}pspnet_model_checkpoint")

Load pretrained Mask RCNN

In [15]:
os.chdir("..")
!git clone https://www.github.com/matterport/Mask_RCNN.git
os.chdir("Mask_RCNN")
!wget --quiet https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5

fatal: destination path 'Mask_RCNN' already exists and is not an empty directory.


In [0]:
from mrcnn import visualize
import mrcnn.model as modellib
from mrcnn import utils
from mrcnn.config import Config

In [0]:
class Fashionista(utils.Dataset):
  def __init__(self, df):
    super().__init__(self)
    
    for i in range(NUM_CLASSES - 1):
        self.add_class("fashionista", i + 1, label_names[i])
    
    for i, img_segments in enumerate(get_images(df)):
        self.add_image("fashionista", 
                      image_id=i, 
                      path=f"../{data_path}{NUM_IMGS}images/{img_segments['ImageId'].iloc[0]}", 
                      labels=img_segments["ClassId"],
                      annotations=img_segments["EncodedPixels"], 
                      height=img_segments["Height"].iloc[0], 
                      width=img_segments["Width"].iloc[0],
                      img_segments=img_segments)
        
  def load_image(self, img_id):
    img = cv2.imread(self.image_info[img_id]["path"])
    return cv2.resize(img, 
                      (IMG_SIZE, IMG_SIZE), 
                      interpolation=cv2.INTER_NEAREST)

      
  def load_mask(self, image_id):
    info = self.image_info[image_id]
    labeled = segments2labeled(info["img_segments"])
    mask, labels, _ = labeled2masked(labeled)
    return mask, labels

In [0]:
dataset = Fashionista(df)
dataset.prepare()

In [0]:
class FashionConfig(Config):
    NAME = "fashionista"
    NUM_CLASSES = NUM_CLASSES 

    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    
    IMAGE_MIN_DIM = IMG_SIZE
    IMAGE_MAX_DIM = IMG_SIZE 
    STEPS_PER_EPOCH = 1000
    
config = FashionConfig()
# config.display()
mrcnn_model = modellib.MaskRCNN(mode="training", config=config, 
                                model_dir="../" + checkpoints_path)

mrcnn_model.load_weights("mask_rcnn_coco.h5", by_name=True, exclude=[
    "mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

In [20]:
mrcnn_model.train(dataset, dataset,
            learning_rate=1e-2,
            epochs=1,
            layers="all")


Starting at epoch 0. LR=0.01

Checkpoint Path: ../drive/My Drive/test_task/checkpoints/fashionista20191215T1553/mask_rcnn_fashionista_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branch2b      

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multi

Epoch 1/1
1000/1000 [==============================] - 358s 358ms/step - loss: 1.6248 - rpn_class_loss: 0.0598 - rpn_bbox_loss: 1.0193 - mrcnn_class_loss: 0.1484 - mrcnn_bbox_loss: 0.2334 - mrcnn_mask_loss: 0.1639 - val_loss: 0.9484 - val_rpn_class_loss: 0.0081 - val_rpn_bbox_loss: 0.7091 - val_mrcnn_class_loss: 0.0400 - val_mrcnn_bbox_loss: 0.1030 - val_mrcnn_mask_loss: 0.0882


In [0]:
os.chdir("..")